### Import Libraries

In [126]:
import numpy as np
import pandas as pd
import math
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
import random

### Import Datasets

In [127]:
restaurant_dataset = pd.read_csv('restaurant.csv')
restaurant_dataset

,Alternative,Bar,Fri/Sat,Hungry,Patrons,Price,Rain,Reservation,Type,Eastimate,WillWait
0,Yes,No,No,Yes,Some,$$$,No,Yes,French,0-10,Yes
1,Yes,No,No,Yes,Full,$,No,No,Thai,30-60,No
2,No,Yes,No,No,Some,$,No,No,Burger,0-10,Yes
3,Yes,No,Yes,Yes,Full,$,No,No,Thai,30-10,Yes
4,Yes,No,Yes,No,Full,$$$,No,Yes,French,>60,No
5,No,Yes,No,Yes,Some,$$,Yes,Yes,Italian,0-10,Yes
6,No,Yes,No,No,None,$,Yes,No,Burger,0-10,No
7,No,No,No,Yes,Some,$$,Yes,Yes,Thai,0-10,Yes
8,No,Yes,Yes,No,Full,$,Yes,No,Burger,>60,No
9,Yes,Yes,Yes,Yes,Full,$$$,No,Yes,Italian,30-10,No


In [128]:
all_feature_names = ['Alternative','Bar','Fri/Sat','Hungry','Patrons','Price','Rain','Reservation','Type','Eastimate']
x = restaurant_dataset[all_feature_names]
y = restaurant_dataset['WillWait']
for column_name in all_feature_names:
    x[column_name]= labelencoder.fit_transform(x[column_name])
x['WillWait']=y
df=x

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [129]:
df

,Alternative,Bar,Fri/Sat,Hungry,Patrons,Price,Rain,Reservation,Type,Eastimate,WillWait
0,1,0,0,1,2,2,0,1,1,0,Yes
1,1,0,0,1,0,0,0,0,3,3,No
2,0,1,0,0,2,0,0,0,0,0,Yes
3,1,0,1,1,0,0,0,0,3,2,Yes
4,1,0,1,0,0,2,0,1,1,4,No
5,0,1,0,1,2,1,1,1,2,0,Yes
6,0,1,0,0,1,0,1,0,0,0,No
7,0,0,0,1,2,1,1,1,3,0,Yes
8,0,1,1,0,0,0,1,0,0,4,No
9,1,1,1,1,0,2,0,1,2,1,No


In [130]:
def train_test_split(df, test_size):
    total_index = df.index.tolist()
    index_of_test_data = random.sample(population=total_index, k=test_size)

    test_data = df.loc[index_of_test_data]
    train_data = df.drop(index_of_test_data)
    
    return train_data, test_data

In [131]:
random.seed(0)
train_data, test_data = train_test_split(df, test_size=4)
data_list = train_data.values

In [132]:
data_list

array([[1, 0, 0, 1, 0, 0, 0, 0, 3, 3, 'No'],
       [0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 'Yes'],
       [1, 0, 1, 1, 0, 0, 0, 0, 3, 2, 'Yes'],
       [0, 1, 0, 1, 2, 1, 1, 1, 2, 0, 'Yes'],
       [0, 0, 0, 1, 2, 1, 1, 1, 3, 0, 'Yes'],
       [0, 1, 1, 0, 0, 0, 1, 0, 0, 4, 'No'],
       [1, 1, 1, 1, 0, 2, 0, 1, 2, 1, 'No'],
       [0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 'No']], dtype=object)

### Necessary Functions

In [133]:
def is_pure(dataset):
    '''
    if the catagory of the output is 1 then this function will return True
    '''
    output_col = dataset[:, -1]
    individual_unique_output = np.unique(output_col)

    if len(individual_unique_output) != 1:
        return False
    else:
        return True

In [134]:
def plurality_value(dataset):
    output_col = dataset[:, -1]
    individual_unique_output, counts_individual_unique_output = np.unique(output_col, return_counts=True)
    index = counts_individual_unique_output.argmax()
    label_with_highest_count = individual_unique_output[index]
    
    return label_with_highest_count

In [135]:
def split_column_data(dataset, column_no, value):
    
    split_column_values = dataset[:, column_no]

    type_of_feature = TYPE_OF_FEATURE[column_no]
    

    less_than_split_value = dataset[split_column_values == value]
    grater_than_split_value = dataset[split_column_values != value]
    
    return less_than_split_value, grater_than_split_value

In [136]:
def entropy(dataset):
    output_column = dataset[:, -1]
    rows, unique_counts = np.unique(output_column, return_counts=True)

    probabilities = unique_counts / unique_counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
    return entropy

In [137]:
def entropy_sum(less_than_split_value, grater_than_split_value):
    
    n = len(less_than_split_value) + len(grater_than_split_value)
    p_less_than_split_value = len(less_than_split_value) / n
    p_grater_than_split_value = len(grater_than_split_value) / n

    total_entropy =  (p_less_than_split_value * entropy(less_than_split_value) 
                      + p_grater_than_split_value * entropy(grater_than_split_value))
    
    return total_entropy

In [138]:
def find_best_split(dataset, split):
    
    total_entropy = 111
    for index in split:
        for value in split[index]:
            less_than_split_value, grater_than_split_value = split_column_data(dataset,index,value)
            current_total_entropy = entropy_sum(less_than_split_value, grater_than_split_value)

            if current_total_entropy <= total_entropy:
                total_entropy = current_total_entropy
                best_column = index
                best_value = value
    
    return best_column, best_value

In [139]:
def decision_tree_Learning_algorithm(df, counter=0, min_samples=2, max_depth=5):

    if counter == 0:
        global column_name, TYPE_OF_FEATURE
        column_name = df.columns
        TYPE_OF_FEATURE=[]
        treshold_val = 10
        for column in df.columns:
            if column != "WillWait":
                unique_values = df[column].unique()
                example_value = unique_values[0]

                if (isinstance(example_value, str)) or (len(unique_values) <= treshold_val):
                    TYPE_OF_FEATURE.append("categorical")

        data = df.values
    else:
        data = df           
   

    if (is_pure(data)) or (len(data) < min_samples) or (counter == max_depth):
        label_with_highest_count = plurality_value(data)
        return label_with_highest_count
  
   
    else:    
        counter += 1
        split={}
        rows,columns = data.shape
        for column_index in range(columns - 1):          # not taking the output column
            column_values = data[:, column_index]
            unique_column_values = np.unique(column_values)
            split[column_index] = unique_column_values
        
        
        split_column1, split_value = find_best_split(data, split)
        less_than_split_value, grater_than_split_value = split_column_data(data, split_column1, split_value)
        
        
        if len(less_than_split_value) == 0 or len(grater_than_split_value) == 0:
            label_with_highest_count = plurality_value(data)
            return label_with_highest_count
        
       
        feature_name = column_name[split_column1]
        type_of_feature = TYPE_OF_FEATURE[split_column1]

        query = f"{feature_name} = {split_value}"
        
        
        sub_tree = {query: []}

        yes = decision_tree_Learning_algorithm(less_than_split_value, counter, min_samples, max_depth)
        no = decision_tree_Learning_algorithm(grater_than_split_value, counter, min_samples, max_depth)

        
        if yes == no:
            sub_tree = yes
        else:
            sub_tree[query].append(yes)
            sub_tree[query].append(no)
        
        return sub_tree

In [140]:
decision_tree = decision_tree_Learning_algorithm(train_data, max_depth=5)
print(decision_tree)

{'Patrons = 2': ['Yes', {'Eastimate = 2': ['Yes', 'No']}]}


In [141]:
def predict1(ck, tree):
    question = list(tree.keys())[0]
    column_name, symbol, value = question.split(" ")


    if str(ck[column_name]) == value:
        query = tree[question][0]
    else:
        query = tree[question][1]

 
    if not isinstance(query, dict):
        return query

    else:
        sub_tree = query
        return predict1(ck, sub_tree)

## Accuracy

In [142]:
def calculate_accuracy(df, tree):

    df["prediction"] = df.apply(predict1, axis=1, args=(tree,))
    df["prediction_correct"] = df["prediction"] == df["WillWait"] #if rows of 'prediction' & 'prediction_correct'(of test_data) both column matches then 'prediction_correct' will be true otherwise false
    accuracy = df["prediction_correct"].mean()
    return accuracy

In [143]:
accuracy = calculate_accuracy(test_data, decision_tree)
accuracy

0.75

## prediction Function

In [144]:
def predict(li):
    test_sample=pd.DataFrame([li], columns = all_feature_names).iloc[0]
    return predict1(test_sample, decision_tree)

In [145]:
data_list

array([[1, 0, 0, 1, 0, 0, 0, 0, 3, 3, 'No'],
       [0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 'Yes'],
       [1, 0, 1, 1, 0, 0, 0, 0, 3, 2, 'Yes'],
       [0, 1, 0, 1, 2, 1, 1, 1, 2, 0, 'Yes'],
       [0, 0, 0, 1, 2, 1, 1, 1, 3, 0, 'Yes'],
       [0, 1, 1, 0, 0, 0, 1, 0, 0, 4, 'No'],
       [1, 1, 1, 1, 0, 2, 0, 1, 2, 1, 'No'],
       [0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 'No']], dtype=object)

In [146]:
predict([1, 0, 0, 1, 0, 0, 0, 0, 3, 3])

'No'